## **Implementando la tokenización**

Los tokenizadores son herramientas esenciales en el procesamiento del lenguaje natural que descomponen el texto en unidades más pequeñas llamadas tokens. Estos tokens pueden ser palabras, caracteres o subpalabras, haciendo que un texto complejo sea comprensible para las computadoras. Al dividir el texto en partes manejables, los tokenizadores permiten que las máquinas procesen y analicen el lenguaje humano, impulsando diversas aplicaciones relacionadas con el lenguaje como la traducción, el análisis de sentimientos y los chatbots. Esencialmente, los tokenizadores cierran la brecha entre el lenguaje humano y la comprensión de la máquina.


#### **Configuración**


Para este cuaderno, utilizarás las siguientes librerías:

* [**nltk**](https://www.nltk.org/) o Natural Language Toolkit, se empleará para tareas de gestión de datos. Ofrece herramientas y recursos integrales para procesar texto en lenguaje natural, lo que la hace una opción valiosa para tareas como el preprocesamiento y análisis de texto.

* [**spaCy**](https://spacy.io/) es una librería de software de código abierto para el procesamiento avanzado del lenguaje natural en Python. spaCy es reconocido por su velocidad y precisión al procesar grandes volúmenes de datos textuales.

* [**BertTokenizer**](https://huggingface.co/docs/transformers/main_classes/tokenizer#berttokenizer) forma parte de la librería Hugging Face Transformers, una librería popular para trabajar con modelos de lenguaje preentrenados de última generación. BertTokenizer está diseñado específicamente para tokenizar texto según las especificaciones del modelo BERT.

* [**XLNetTokenizer**](https://huggingface.co/docs/transformers/main_classes/tokenizer#xlnettokenizer) es otro componente de la librería Hugging Face Transformers. Está adaptado para tokenizar texto de acuerdo con los requerimientos del modelo XLNet.

* [**torchtext**](https://pytorch.org/text/stable/index.html) es parte del ecosistema de PyTorch, para manejar diversas tareas de procesamiento del lenguaje natural. Simplifica el proceso de trabajar con datos textuales y provee funcionalidades para el preprocesamiento de datos, tokenización, gestión de vocabulario y agrupación en lotes.


#### Instalación de librerías requeridas


In [ ]:
#!pip install nltk
#!pip install transformers
#!pip install sentencepiece
#!pip install spacy
#!pip install numpy==1.24
#!python -m spacy download en_core_web_sm
#!python -m spacy download de_core_news_sm
#!pip install numpy scikit-learn
#!pip install torch==2.0.1
#!pip install torchtext==0.15.2

#### Importación de librerías requeridas

_Se recomienda importar todas las librerías requeridas en un solo lugar (aquí):_


In [ ]:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')
import spacy
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.util import ngrams
from transformers import BertTokenizer
from transformers import XLNetTokenizer

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

#### ¿Qué es un tokenizador y por qué lo usamos?

Los tokenizadores juegan un papel fundamental en el procesamiento del lenguaje natural, segmentando el texto en unidades más pequeñas conocidas como tokens. Estos tokens se transforman posteriormente en representaciones numéricas llamadas índices de tokens, que son utilizados directamente por los algoritmos de aprendizaje profundo.
<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%201.png" width="50%" alt="Image Description">
</center>


#### Tipos de tokenizador

La representación significativa puede variar dependiendo del modelo en uso. Diversos modelos emplean algoritmos de tokenización distintos, y se cubrirán ampliamente los siguientes enfoques. Transformar el texto en valores numéricos puede parecer sencillo al principio, pero abarca varias consideraciones que se deben tener en cuenta.
<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%202.png" width="50%" alt="Image Description">
</center>


### Tokenizador basado en palabras

#### nltk

Como su nombre indica, se trata de dividir el texto basándose en palabras. Existen diferentes reglas para los tokenizadores basados en palabras, como dividir por espacios o por puntuación. Cada opción asigna un ID específico a la palabra dividida. Aquí se utiliza el ```word_tokenize``` de nltk


In [ ]:
texto = "This is a sample sentence for word tokenization."
tokens = word_tokenize(texto)
print(tokens)

Librerías generales como nltk y spaCy a menudo dividen palabras como "don't" y "couldn't", que son contracciones, en palabras individuales separadas. **No existe una regla universal, y cada librería tiene sus propias reglas de tokenización para tokenizadores basados en palabras**. Sin embargo, la pauta general es preservar el formato de entrada después de la tokenización para que coincida con la forma en que se entrenó el modelo.


In [ ]:
# Esto muestra word_tokenize de la librería nltk

texto = "I couldn't help the dog. Can't you do it? Don't be afraid if you are."
tokens = word_tokenize(texto)
print(tokens)

In [ ]:
# Esto muestra el uso del tokenizador de 'spaCy' con la función get_tokenizer de torchtext

texto = "I couldn't help the dog. Can't you do it? Don't be afraid if you are."
nlp = spacy.load("en_core_web_sm")
doc = nlp(texto)

# Creando una lista de tokens e imprimiéndola
token_list = [token.text for token in doc]
print("Tokens:", token_list)

# Mostrando detalles de cada token
for token in doc:
    print(token.text, token.pos_, token.dep_)

Explicación de algunas líneas:
- I PRON nsubj: "I" es un pronombre (PRON) y es el sujeto nominal (nsubj) de la oración.
- help VERB ROOT: "help" es un verbo (VERB) y es la acción principal (ROOT) de la oración.
- afraid ADJ acomp: "afraid" es un adjetivo (ADJ) y es un complemento adjetival (acomp) que aporta más información sobre un estado o cualidad relacionado con el verbo.


El problema con este algoritmo es que las palabras con significados similares serán asignadas con IDs diferentes, lo que resulta en que se traten como palabras completamente separadas con significados distintos. Por ejemplo, *Unicorns* es la forma plural de *Unicorn*, pero un tokenizador basado en palabras las tokenizaría como dos palabras separadas, lo que podría causar que el modelo no reconozca su relación semántica.


In [ ]:
texto = "Unicorns are real. I saw a unicorn yesterday."
token = word_tokenize(texto)
print(token)

Cada palabra se divide en un token, lo que conduce a un aumento significativo en el vocabulario total del modelo. Cada token se asigna a un vector grande que contiene los significados de la palabra, resultando en parámetros de modelo grandes.


Los idiomas generalmente tienen una gran cantidad de palabras, por lo que los vocabularios basados en ellas siempre serán extensos. Sin embargo, el número de caracteres en un idioma siempre es menor en comparación con el número de palabras. A continuación, exploraremos los tokenizadores basados en caracteres.


#### Tokenizador basado en caracteres

Como su nombre indica, la tokenización basada en caracteres implica dividir el texto en caracteres individuales. La ventaja de utilizar este enfoque es que los vocabularios resultantes son intrínsecamente pequeños. Además, dado que los idiomas tienen un conjunto limitado de caracteres, el número de tokens fuera del vocabulario también es limitado, reduciendo el desperdicio de tokens.

Por ejemplo:
Texto de entrada: `This is a sample sentence for tokenization.`

Salida de tokenización basada en caracteres: `['T', 'h', 'i', 's', 'i', 's', 'a', 's', 'a', 'm', 'p', 'l', 'e', 's', 'e', 'n', 't', 'e', 'n', 'c', 'e', 'f', 'o', 'r', 't', 'o', 'k', 'e', 'n', 'i', 'z', 'a', 't', 'i', 'o', 'n', '.']`

Sin embargo, es importante notar que la tokenización basada en caracteres tiene sus limitaciones. Los caracteres individuales pueden no transmitir la misma información que las palabras completas, y la longitud total de los tokens aumenta significativamente, lo que podría causar problemas con el tamaño del modelo y una pérdida de rendimiento.


Has explorado las limitaciones de los métodos de tokenización basados en palabras y caracteres. Para aprovechar las ventajas de ambos enfoques, los transformers emplean la tokenización basada en subpalabras, que se discutirá a continuación.


### Tokenizador basado en subpalabras

El tokenizador basado en subpalabras permite que las palabras de uso frecuente permanezcan sin dividir, mientras que descompone las palabras poco frecuentes en subpalabras significativas. Técnicas como [SentencePiece](https://github.com/google/sentencepiece) o [WordPiece](https://paperswithcode.com/method/wordpiece) se utilizan comúnmente para la tokenización de subpalabras. Estos métodos aprenden unidades de subpalabras a partir de un corpus de texto dado, identificando prefijos, sufijos y raíces comunes como tokens de subpalabras basados en su frecuencia de aparición. Este enfoque ofrece la ventaja de representar una gama más amplia de palabras y adaptarse a los patrones específicos del lenguaje dentro de un corpus de texto.

En ambos ejemplos a continuación, las palabras se dividen en subpalabras, lo que ayuda a preservar la información semántica asociada con la palabra completa. Por ejemplo, "Unhappiness" se divide en "un" y "happiness", las cuales pueden aparecer como subpalabras independientes. Cuando combinamos estas subpalabras individuales, forman "unhappiness", que conserva su contexto significativo. Este enfoque ayuda a mantener la información general y el significado semántico de las palabras.

<center>
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-AI0201EN-Coursera/images/Tokenization%20lab%20Diagram%203.png" width="50%" alt="Image Description">
</center>


#### WordPiece

Inicialmente, WordPiece inicializa su vocabulario para incluir cada carácter presente en los datos de entrenamiento y aprende progresivamente un número especificado de reglas de fusión. WordPiece no selecciona el par de símbolos más frecuente, sino aquel que maximiza la probabilidad de los datos de entrenamiento al añadirse al vocabulario. En esencia, WordPiece evalúa lo que sacrifica al fusionar dos símbolos para asegurar que sea un esfuerzo que valga la pena.

Ahora, el tokenizador WordPiece está implementado en BertTokenizer.
Ten en cuenta que BertTokenizer trata las palabras compuestas como tokens separados.


In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
tokenizer.tokenize("IBM taught me tokenization.")

A continuación, se muestra un desglose de la salida:
- 'ibm': "IBM" se tokeniza como 'ibm'. BERT convierte los tokens a minúsculas, ya que no conserva la información de mayúsculas cuando se utiliza el modelo "bert-base-uncased".
- 'taught', 'me', '.': Estos tokens son iguales a las palabras o puntuaciones originales, solo que en minúsculas (excepto la puntuación).
- 'token', '##ization': "Tokenization" se divide en dos tokens. "Token" es una palabra completa, y "##ization" es una parte de la palabra original. El "##" indica que "ization" debe conectarse de nuevo a "token" al realizar la detokenización (transformar tokens de vuelta a palabras).


#### Unigram y SentencePiece

Unigram es un método para dividir palabras o texto en piezas más pequeñas. Lo logra comenzando con una lista amplia de posibilidades y reduciéndola gradualmente según la frecuencia con la que aparecen esas piezas en el texto. Este enfoque ayuda a una tokenización eficiente del texto.

SentencePiece es una herramienta que toma el texto, lo divide en partes más pequeñas y manejables, asigna IDs a estos segmentos y se asegura de hacerlo de manera consistente. En consecuencia, si utilizas SentencePiece en el mismo texto de manera repetida, obtendrás consistentemente las mismas subpalabras e IDs.

Unigram y SentencePiece trabajan juntos implementando el método de tokenización de subpalabras de unigrama dentro del marco de SentencePiece. SentencePiece maneja la segmentación de subpalabras y la asignación de IDs, mientras que los principios de Unigrama guían el proceso de reducción del vocabulario para crear una representación más eficiente de los datos textuales. Esta combinación es especialmente valiosa para diversas tareas de NLP en las que la tokenización de subpalabras puede mejorar el rendimiento de los modelos de lenguaje.


In [ ]:
tokenizer = XLNetTokenizer.from_pretrained("xlnet-base-cased")
tokenizer.tokenize("IBM taught me tokenization.")

Esto es lo que sucede con cada token:
- '▁IBM': El "▁" (a menudo referido como "carácter de espacio") antes de "IBM" indica que este token está precedido por un espacio en el texto original. "IBM" se mantiene tal cual porque es reconocido como un token completo por XLNet y preserva la capitalización, ya que se está utilizando el modelo "xlnet-base-cased".
- '▁taught', '▁me', '▁token': De manera similar, estos tokens están prefijados con "▁" para indicar que son palabras nuevas precedidas por un espacio en el texto original, preservando la palabra completa y manteniendo la capitalización original.
- 'ization': A diferencia de "BertTokenizer", "XLNetTokenizer" no utiliza "##" para indicar tokens de subpalabras. "ization" aparece como un token propio sin prefijo porque sigue directamente a la palabra anterior "token" sin un espacio en el texto original.
- '.': El punto se tokeniza como un token separado ya que la puntuación se trata por separado.


### Tokenización con PyTorch 

En PyTorch, especialmente con la librería `torchtext`, el tokenizador descompone el texto de un conjunto de datos en palabras o subpalabras individuales, facilitando su conversión a un formato numérico. Después de la tokenización, el vocabulario asigna a estos tokens enteros únicos, permitiendo que sean utilizados en redes neuronales. Este proceso es vital porque los modelos de aprendizaje profundo operan con datos numéricos y no pueden procesar texto sin formato directamente. 

Así, la tokenización y la asignación del vocabulario sirven como puente entre el texto legible para humanos y los datos numéricos operables por la máquina. Considera el siguiente conjunto de datos:


In [ ]:
dataset = [
    (1,"Introduction to NLP"),
    (2,"Basics of PyTorch"),
    (1,"NLP Techniques for Text Classification"),
    (3,"Named Entity Recognition with PyTorch"),
    (3,"Sentiment Analysis using PyTorch"),
    (3,"Machine Translation with PyTorch"),
    (1," NLP Named Entity,Sentiment Analysis,Machine Translation "),
    (1," Machine Translation with NLP "),
    (1," Named Entity vs Sentiment Analysis  NLP ")]


La siguiente línea importa la función ```get_tokenizer``` desde el módulo ```torchtext.data.utils```. En la librería torchtext, la función ```get_tokenizer``` se utiliza para obtener un tokenizador por nombre. Proporciona soporte para una variedad de métodos de tokenización, incluyendo la división básica de cadenas, y devuelve varios tokenizadores según el argumento que se le pase.


In [ ]:
from torchtext.data.utils import get_tokenizer

In [ ]:
tokenizer = get_tokenizer("basic_english")

Se aplica el tokenizador al conjunto de datos. Nota: Si se selecciona ```basic_english```, retorna la función ```_basic_english_normalize()```, que normaliza la cadena primero y luego la divide por espacios.


In [ ]:
tokenizer(dataset[0][1])

### Índices de tokens
Se representan las palabras como números, ya que los algoritmos de PLN pueden procesar y manipular números de manera más eficiente y rápida que el texto sin procesar. Se utiliza la función **```build_vocab_from_iterator```**, cuyo resultado se denomina típicamente 'índices de tokens' o simplemente 'índices'. Estos índices representan las representaciones numéricas de los tokens en el vocabulario.

La función **```build_vocab_from_iterator```**, cuando se aplica a una lista de tokens, asigna un índice único a cada token basado en su posición en el vocabulario. Estos índices sirven como una forma de representar los tokens en un formato numérico que puede ser procesado fácilmente por modelos de aprendizaje automático.

Por ejemplo, dado un vocabulario con tokens `["apple", "banana", "orange"]`, los índices correspondientes podrían ser `[0, 1, 2]`, donde "apple" se representa con el índice 0, "banana" con el 1, y "orange" con el 2.

**```dataset```** es un iterable. Por lo tanto, se utiliza una función generadora `yield_tokens` para aplicar el **```tokenizer```**. El propósito de la función generadora **```yield_tokens```** es producir textos tokenizados uno a la vez. En lugar de procesar todo el conjunto de datos y devolver todos los textos tokenizados de una vez, la función generadora procesa y produce cada texto tokenizado individualmente a medida que se solicita. 

El proceso de tokenización se realiza de forma perezosa, lo que significa que el siguiente texto tokenizado se genera solo cuando es necesario, ahorrando memoria y recursos computacionales.


In [ ]:
def yield_tokens(data_iter):
    # Función generadora para producir tokens a partir del conjunto de datos
    for _, text in data_iter:
        yield tokenizer(text)

In [ ]:
# Crear un iterador llamado my_iterator utilizando la función generadora yield_tokens
my_iterator = yield_tokens(dataset)

Esto crea un iterador llamado **```my_iterator```** usando la función generadora. Para comenzar la evaluación del generador y recuperar los valores, puedes iterar sobre **```my_iterator```** utilizando un bucle for o recuperar valores usando la función **```next()```**.


In [ ]:
# Obtener el siguiente elemento del iterador my_iterator
next(my_iterator)

Construyes un vocabulario a partir de los textos tokenizados generados por la función generadora **```yield_tokens```**, que procesa el conjunto de datos. La función **```build_vocab_from_iterator()```** construye el vocabulario, incluyendo un token especial `unk` para representar las palabras fuera del vocabulario. 

#### Fuera del vocabulario (OOV)
Cuando los datos de texto se tokenizan, puede haber palabras que no están presentes en el vocabulario porque son raras o no vistas durante el proceso de construcción del vocabulario. Al encontrar dichas palabras fuera del vocabulario durante tareas reales de procesamiento del lenguaje, como la generación de texto o el modelado de lenguaje, el modelo puede usar el token `<unk>` para representarlas.

Por ejemplo, si la palabra "apple" está presente en el vocabulario, pero "pineapple" no lo está, "apple" se usará normalmente en el texto, pero "pineapple" (al ser una palabra fuera del vocabulario) se reemplazaría por el token `<unk>`.

Al incluir el token `<unk>` en el vocabulario, se proporciona una forma consistente de manejar palabras fuera del vocabulario en tu modelo de lenguaje u otras tareas de procesamiento del lenguaje natural.


In [ ]:
vocab = build_vocab_from_iterator(yield_tokens(dataset), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

Este código demuestra cómo obtener una oración tokenizada de un iterador, convertir sus tokens en índices utilizando un vocabulario provisto, y luego imprimir tanto la oración original como sus índices correspondientes.


In [ ]:
def get_tokenized_sentence_and_indices(iterator):
    tokenized_sentence = next(iterator)  # Obtener la siguiente oración tokenizada
    token_indices = [vocab[token] for token in tokenized_sentence]  # Obtener los índices de los tokens
    return tokenized_sentence, token_indices

tokenized_sentence, token_indices = get_tokenized_sentence_and_indices(my_iterator)
next(my_iterator)

print("Oracion tokenizadas:", tokenized_sentence)
print("Índices de tokens:", token_indices)

Usando las líneas de código proporcionadas anteriormente en un ejemplo sencillo, demuestra la tokenización y la construcción del vocabulario en PyTorch.


In [ ]:
lineas = ["IBM taught me tokenization", 
         "Special tokenizers are ready and they will blow your mind", 
         "just saying hi!"]

special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

tokenizer_en = get_tokenizer('spacy', language='en_core_web_sm')

tokens = []
max_length = 0

for linea in lineas:
    tokenized_line = tokenizer_en(linea)
    tokenized_line = ['<bos>'] + tokenized_line + ['<eos>']
    tokens.append(tokenized_line)
    max_length = max(max_length, len(tokenized_line))

for i in range(len(tokens)):
    tokens[i] = tokens[i] + ['<pad>'] * (max_length - len(tokens[i]))

print("Líneas después de agregar tokens especiales:\n", tokens)

# Construir vocabulario sin unk_init
vocab = build_vocab_from_iterator(tokens, specials=['<unk>'])
vocab.set_default_index(vocab["<unk>"])

# Vocabulario e IDs de tokens
print("Vocabulary:", vocab.get_itos())
print("Token IDs for 'tokenization':", vocab.get_stoi())

Desglosemos la salida:
1. **Tokens especiales**:
- Token: "`<unk>`", Índice: 0: `<unk>` significa "desconocido" y representa las palabras que no se vieron durante la construcción del vocabulario, usualmente durante la inferencia en texto nuevo.
- Token: "`<pad>`", Índice: 1: `<pad>` es un token de "relleno" utilizado para hacer que las secuencias de palabras tengan la misma longitud al agruparlas en lotes.
- Token: "`<bos>`", Índice: 2: `<bos>` es el acrónimo de "inicio de secuencia" y se usa para denotar el comienzo de una secuencia de texto.
- Token: "`<eos>`", Índice: 3: `<eos>` es el acrónimo de "fin de secuencia" y se usa para denotar el final de una secuencia de texto.

2. **Tokens de palabras**:
El resto de los tokens son palabras o signos de puntuación extraídos de las oraciones proporcionadas, cada uno asignado a un índice único:
- Token: "IBM", Índice: 5
- Token: "taught", Índice: 16
- Token: "me", Índice: 12
    ... y así sucesivamente.
    
3. **Vocabulario**:
Representa el número total de tokens en las oraciones sobre las cuales se construyó el vocabulario.
    
4. **IDs de tokens para 'tokenization'**:
Representa los IDs de tokens asignados en el vocabulario donde un número representa su presencia en la oración.


In [ ]:
nueva_linea = "I learned about embeddings and attention mechanisms."

# Tokenizar la nueva línea
tokenized_new_line = tokenizer_en(nueva_linea)
tokenized_new_line = ['<bos>'] + tokenized_new_line + ['<eos>']

# Rellenar la nueva línea para que coincida con la longitud máxima de las líneas anteriores
new_line_padded = tokenized_new_line + ['<pad>'] * (max_length - len(tokenized_new_line))

# Convertir tokens a IDs y manejar palabras desconocidas
new_line_ids = [vocab[token] if token in vocab else vocab['<unk>'] for token in new_line_padded]

# Ejemplo de uso
print("Id de tokens para nueva linea:", new_line_ids)

Desglosemos la salida:

1. **Tokens especiales**:
- Token: "`<unk>`", Índice: 0: `<unk>` significa "desconocido" y representa las palabras que no se vieron durante la construcción del vocabulario, usualmente durante la inferencia en texto nuevo.
- Token: "`<pad>`", Índice: 1: `<pad>` es un token de "relleno" utilizado para hacer que las secuencias de palabras tengan la misma longitud al agruparlas en lotes.
- Token: "`<bos>`", Índice: 2: `<bos>` es el acrónimo de "inicio de secuencia" y se usa para denotar el comienzo de una secuencia de texto.
- Token: "`<eos>`", Índice: 3: `<eos>` es el acrónimo de "fin de secuencia" y se usa para denotar el final de una secuencia de texto.

2. El token **`and`** es reconocido en la oración y se le asigna **`token_id` - 7**.


#### Ejercicio: Tokenización comparativa de texto y análisis de rendimiento
- Objetivo: Evaluar y comparar las capacidades de tokenización de cuatro librerías de PLN diferentes (`nltk`, `spaCy`, `BertTokenizer` y `XLNetTokenizer`) analizando la frecuencia de las palabras tokenizadas y midiendo el tiempo de procesamiento de cada herramienta usando `datetime`.
- El texto para la tokenización es el siguiente:


In [ ]:
texto = """
Going through the world of tokenization has been like walking through a huge maze made of words, symbols, and meanings. Each turn shows a bit more about the cool ways computers learn to understand our language. And while I'm still finding my way through it, the journey’s been enlightening and, honestly, a bunch of fun.
Eager to see where this learning path takes me next!"
"""

# Contar y mostrar tokens y su frecuencia
#from collections import Counter
#def show_frequencies(tokens, method_name):
#    print(f"{method_name} Token Frequencies: {dict(Counter(tokens))}\n")

In [ ]:
## Tu respuesta
